#### Модель sklearn Gradient Boosting Regressor

In [8]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
import sqlite3
import pickle

In [9]:
#Данные, из total.csv и coords.csv в БД apartments.db


con = sqlite3.connect("/users/andrejmironov/Documents/flask-learn/instance/apartments.db")
cur = con.cursor()



In [12]:
#Формирует DF, объединение по адресу

total = total = pd.read_sql_query(

    "select * from total t left join coords c on t.address == c.address ", 
    con
)






total.drop(columns=['price', 'street', 'house', 'address', 'address', 'latitude', 'longitude', 'full_address'], inplace=True)

total = total.query("newbuilding == 'Вторичка'")






#удаляю выбросы - 1,5 межквантильных размаха из колонки со средней стоимостью метра
Q1 = total['kv'].quantile(0.25)
Q3 = total['kv'].quantile(0.75)
IQR = Q3 - Q1
a = total[(total['kv'] < Q1-1.5*IQR ) | (total['kv'] > Q3+1.5*IQR)]['kv'].reset_index()
    
#список с исключениями
a = list(a.kv)
#исключаю выбросы
total= total.query("kv != @a and material != '0' and hight != 1")



#меняем категориальные переменные
column_list = []
for c in list(total.columns):
    if total.dtypes['{}'.format(c)] == 'object':
        column_list.append(c)
total = pd.get_dummies(data=total, columns=column_list).drop(columns=['index'])


In [13]:
#Формирует тестовую и обучающую выборки, метрика - RMSE
X_train,X_test, y_train, y_test = train_test_split(total.drop('kv',axis=1),
                                                        total.kv,
                                                        shuffle = True,
                                                        random_state = 42,
                                                        test_size = 0.03)




dtr = GradientBoostingRegressor(n_estimators=200)
dtr.fit(X_train, y_train)
print('Ошибка на обучающей выборке', (mean_squared_error(y_train, dtr.predict(X_train))**0.5))
pred = dtr.predict(X_test)
print('Ошибка на тесте', (mean_squared_error(y_test, pred)**0.5))


Ошибка на обучающей выборке 16863.534118427575
Ошибка на тесте 19839.92649886385


In [348]:
#Для колонки newbuilding создает файлы, далее используется для предсказания стоимости

with open('data.pickle', 'wb') as f:
    pickle.dump(dtr, f)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
a = total.groupby('transport_accessibility')['city_area'].count().reset_index()
fig, ax = plt.subplots()
ax.barh(a.transport_accessibility, a.city_area, edgecolor="white", linewidth=0.7)

In [ ]:
a = total.groupby('city_area')['rooms'].count().reset_index()
fig, ax = plt.subplots()
ax.barh(a.city_area, a.rooms, edgecolor="white", linewidth=0.7)

In [ ]:
a = total.groupby('metro')['rooms'].count().reset_index()
a.metro.replace('0', 'Нет', inplace=True)
fig, ax = plt.subplots()
ax.pie(a.rooms, labels=a.metro)

In [ ]:
a = total.groupby('material')['city_area'].count().reset_index()
fig, ax = plt.subplots()
ax.barh(a.material, a.city_area, edgecolor="white", linewidth=0.7)

In [ ]:
a = total.groupby('hight')['city_area'].count().reset_index()
fig, ax = plt.subplots()
ax.bar(a.hight, a.city_area, edgecolor="white", linewidth=0.7)

In [ ]:
a = total.groupby('rooms')['city_area'].count().reset_index()
fig, ax = plt.subplots()
ax.bar(a.rooms, a.city_area, edgecolor="white", linewidth=0.7)

In [ ]:
a = total.query("hight > 5 and hight <= 9").groupby('floor')['city_area'].count().reset_index()
fig, ax = plt.subplots()
ax.bar(a.floor, a.city_area, edgecolor="white", linewidth=0.7)